In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

/u02/thanh/workplace/plasma/.env/RnD/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
def input_handler(i):
    return [i] * 2

In [5]:
out_queue= ppq.TransferQueue().register_callback(print)
temp = partials(print, 'test')

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), input_handler, distributor),
            (input_handler, compute, ppq.ProcessQueue(5)),
                (compute, print, ppq.TransferQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ProcessQueue(5)),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ProcessQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=140622745434976)]

  |->(<function input_handler at 0x7fe6b06a4860>, id=140628778960992)-IteratorDistributor

    |->[ProcessQueue(name=None, runner=5, id=140622745435312)]

      |->(<function compute at 0x7fe548b80ae0>, id=140622744259296)

        |->[TransferQueue(name=None, runner=1, id=140622744053840)]

          |->(<built-in function print>, id=140628978803760)

        |->[TransferQueue(name=None, runner=1, id=140628928218848)]

        |->[ProcessQueue(name=None, runner=5, id=140622744055440)]

          |->(<function compute2 at 0x7fe548b80b80>, id=140622744259456)

            |->[TransferQueue(name=None, runner=1, id=140622744053840)]

              |->(<built-in function print>, id=140628978803760)...

            |->[TransferQueue(name=None, runner=1, id=140628928218848)]

            |->[ProcessQueue(name=None, runner=1, id=140622744055760)]

              |->(ComputeClass, id=140622745434640)

                |->[TransferQueue(name

In [6]:
with flow:
   for i in range(10):
      flow.put(i)

compute: 0
compute: 0
compute: 1
compute: 1
compute: 2
compute: 2
compute: 3
compute: 3
compute: 4
compute2: compute: 0c
compute2: compute: 1c
compute: 4
compute2: compute: 0c
compute2: compute: 2c
compute2: compute: 1c
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
compute: 5
compute: 6
compute: 6
compute: 5
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 7
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
ComputeClass: compute2: compute: 3cd
compute2: compute: 3c
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 7
compute: 8
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 8
compute: 9
ComputeClass: compute2: compute: 6cd
compute2: compute: 6c
compute: 9
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
ComputeClass:

In [7]:
flow.alive()

False

In [8]:
out_queue.run()

compute: 0
compute: 0
compute: 1
compute: 1
compute: 2
compute: 2
compute: 3
compute: 4
compute2: compute: 0c
compute: 3
compute2: compute: 1c
compute: 4
compute2: compute: 0c
compute2: compute: 2c
compute2: compute: 1c
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 1cd
ComputeClass: compute2: compute: 0cd
ComputeClass: compute2: compute: 2cd
ComputeClass: compute2: compute: 1cd
compute: 5
compute: 5
compute: 6
compute: 6
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 7
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 7
compute: 8
ComputeClass: compute2: compute: 5cd
compute2: compute: 5c
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute: 8
compute: 9
compute: 9
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute2: com

TransferQueue(
  name=None,
  num_runner=1,
)

In [9]:
out_queue.release()